# Non stationary covariances

This tutorial aims to show how to handle non stationary covariances for kriging, covariance evaluation, and SPDE approach.

For the covariance based approach (used in the gstlearn function **kriging**) the covariance formulas can be obtain in [this monograph](http://www-personal.umich.edu/~jizhu/jizhu/covar/Stein-Summary.pdf) of M. Stein.

For the SPDE approach, a detailed material can be obtained in the [Phd thesis](https://pastel.hal.science/tel-02499376v1/document) of M. Pereira.

In **gstlearn**, there are two ways to specify the non stationary covariance :
* by providing maps of parameters through a Db which covers the domain
* by providing a function (**AFunctional** structure of gstlearn)

We will explain the two approaches.

We first create a reference model.

In [1]:
import gstlearn as gl
import numpy as np

model =gl.Model.createFromParam(gl.ECov.MATERN,range=10,param=1, sill = 2)

We extract the **CovAniso** structure

In [ ]:
cova = model.getCova(0)
cova

We start by the first approach (non-stationarity defined through a **Db**).

We define a **Db** of parameters named **grid1** for which we fill some columns of values (arbitrary in this tutorial).

In [3]:
grid1 = gl.DbGrid.create([10,10])
grid1["param1"] = 2 * grid1["x1"] +12
grid1["param2"] = 2 * grid1["x2"] +5

Then we can make the different parameters of the covariance non stationary by first attaching the **Db** and then providing the column name of the non stationary parameter. For instance:

In [ ]:
cova.attachNoStatDb(grid1)
cova.makeAngleNoStatDb("param1")
cova.makeRangeNoStatDb("param2",1) #Range parameter in the second direction
cova

Another way of doing this, is to specify the **Db** when you give the specification of the parameter. In that case, you don't need to explicitely attach a **Db** with **attachDbNoStat**.

In [ ]:
model =gl.Model.createFromParam(gl.ECov.MATERN,range=10,param=1, sill = 2)
cova = model.getCova(0)
cova.makeAngleNoStatDb("param1",db = grid1)
cova.makeRangeNoStatDb("param2",1)
cova

In that case, **grid1** is recorded as the reference **Db** for the next parameters.

You can even have different **Db** for different parameters.

In [ ]:

grid2 = gl.DbGrid.create([10,10])
grid2["paramnew"] = 3 * grid2["x2"] +24

cova.makeSillNoStatDb("paramnew",db = grid2)
cova

But the reference **Db** is the first one which has been attached (by **attachNoStatDb** or by the first parameter specification)

In [ ]:
cova.makeRangeNoStatDb("paramnew")

Of course, you have to provide a **Db** to be allowed to give a specification:

In [ ]:
model =gl.Model.createFromParam(gl.ECov.MATERN,range=10,param=1, sill = 2)
cova = model.getCova(0)
cova.makeAngleNoStatDb("param1")
cova

If you give several specifications for a given parameter, you have a warning message and the last specification is kept.

In [ ]:
model =gl.Model.createFromParam(gl.ECov.MATERN,range=10,param=1, sill = 2)
cova = model.getCova(0)
cova.makeAngleNoStatDb("param1",db = grid1)
cova.makeAngleNoStatDb("param2")

You can specify the non stationarity by using the scale instead of the practical range.

In [ ]:
model =gl.Model.createFromParam(gl.ECov.MATERN,range=10,param=1, sill = 2)
cova = model.getCova(0)
cova.makeScaleNoStatDb("param1",db = grid1)
cova

Same king of message if you specify the range after having specified the scale (for the same direction).

In [ ]:
model =gl.Model.createFromParam(gl.ECov.MATERN,range=10,param=1, sill = 2)
cova = model.getCova(0)
cova.makeScaleNoStatDb("param1",db = grid1)
cova.makeRangeNoStatDb("param1",db = grid1)
cova

You can't work in scale for one dimension and in range for another one:

In [ ]:
model =gl.Model.createFromParam(gl.ECov.MATERN,range=10,param=1, sill = 2)
cova = model.getCova(0)
cova.makeScaleNoStatDb("param1",0,db = grid1)
cova.makeRangeNoStatDb("param1",1,db = grid1)
cova

You can make back a parameter stationary

In [ ]:
model =gl.Model.createFromParam(gl.ECov.MATERN,range=10,param=1, sill = 2)
cova = model.getCova(0)
cova.makeScaleNoStatDb("param1",0,db = grid1)
cova.makeScaleStationary(0)
cova

If you make a range stationary whereas the non-stationarity was previously defined for scale, the parameter is made stationary as well.

In [ ]:
model =gl.Model.createFromParam(gl.ECov.MATERN,range=10,param=1, sill = 2)
cova = model.getCova(0)
cova.makeScaleNoStatDb("param1",0,db = grid1)
cova.makeRangeStationary(0)
cova

You can clear all the non-stationarity specifications.

In [ ]:
model =gl.Model.createFromParam(gl.ECov.MATERN,range=10,param=1, sill = 2)
cova = model.getCova(0)
cova.makeScaleNoStatDb("param1",0,db = grid1)
cova.makeAngleNoStatDb("param1",0,db = grid1)
cova.makeStationary()
cova

The non-stationarity is transfered when you clone your **CovAniso**

In [ ]:
model =gl.Model.createFromParam(gl.ECov.MATERN,range=10,param=1, sill = 2)
cova = model.getCova(0)
cova.makeScaleNoStatDb("param1",0,db = grid1)
cova.makeAngleNoStatDb("param1",0,db = grid1)
cova2 = cova.clone()
cova2

You can specify non-stationarities for the terms of the sill matrix in the multivariate setting:

In [ ]:
model =gl.Model.createFromParam(gl.ECov.MATERN,range=10,param=1, 
                                sills = np.array([[2.,1.],[1.,3.]]))
cova = model.getCova(0)
cova.makeSillNoStatDb("param1",0,1,db = grid1)
cova

### Non stationarity by a function

To understand how to play with a function, let's have a look on a part of the **AFunctional** C++ class of the gstlearn library.

We can see that the class is built by using an integer (the space dimension), and that there is mainly to methods.
The first one is pure virtual, that means that the users who want to create a class which inherits from **AFunctional** have to implement this method. The aim of this method is to give the value at a given position (**pos**) specified through a vector of dimension **_ndim**. 

Here is an example of class in python to inherit from the **AFunctional** class.

In [18]:
class MyFunction(gl.AFunctional):
    def __init__(self,ndim):
        super(MyFunction,self).__init__(ndim)
        pass
    
    def getFunctionValue(self, coords):
        return coords[0] + 1

You can then instantiate an object from the class **MyFunction** and use it to define non stationarity:

In [ ]:
myfunc = MyFunction(2)

model =gl.Model.createFromParam(gl.ECov.MATERN,range=10,param=1, 
                                sills = np.array([[2.,1.],[1.,3.]]))
cova = model.getCova(0)
cova.makeSillNoStatFunctional(myfunc,0,1)
cova


You can even mix different types of non-stationarities (with **AFunctional** and with **NoStatDb**):

In [ ]:
model =gl.Model.createFromParam(gl.ECov.MATERN,range=10,param=1, 
                                sills = np.array([[2.,1.],[1.,3.]]))
cova = model.getCova(0)
cova.makeSillNoStatDb("param1",0,0,grid1)
cova.makeSillNoStatFunctional(myfunc,0,1)
cova.makeSillNoStatDb("param1",1,1,grid1)

cova